# Robótica Móvel - Trabalho Prático 1 (Exercícios 5-6)
---
**Nome Completo:** Daniel Terra Gomes

**Matrícula:** 2025702870 

**Programa:** Mestrando do PPGCC  
**Departamento:** Departamento de Ciência da Computação  
**Instituição:** ICEx-UFMG  
**E-mail:** danielterragomes@ufmg.br

**Data:** 11 de setembro de 2025

---

Este notebook implementa os exercícios 5-6 do TP1, focando em:
- Integração de sensores laser (Hokuyo)
- Transformação de dados de sensores para frame global
- Navegação autônoma básica
- Mapeamento incremental com dados de laser

**Atenção:** Use a cena `T1-ex5.ttt` que contém o robô Pioneer P3DX com sensor laser.

In [ ]:
# Imports necessários
import numpy as np

# Import das utils criadas para o TP1
#TODO: import the necessary classes and functions
from robotics_utils import (
    CoppeliaSimConnector,
    SceneAnalyzer,
    wait_for_user_input,
    validate_transformation_matrix,
    HokuyoSensorSim
)

# === Configuration and Constants ===


#TODO: modify it to this exercises: the robot and add its laser sensor /PioneerP3DX/fastHokuyo
# Default object mapping for TP1 scenes
DEFAULT_OBJECT_MAPPING_EX5_6 = {
    'Robot': 'PioneerP3DX',
    'Bill_0': 'Bill[0]',
    'Bill_1': 'Bill[1]',
    'Crate': 'Floor/ConcretBlock',
    'Pillar_0': 'Floor/20cmHighPillar10cm[0]',
    'Pillar_1': 'Floor/20cmHighPillar10cm[1]',
    'Hokuyo': 'PioneerP3DX/fastHokuyo',
    'Table': 'diningTable',
    'Laptop_0': 'diningTable/laptop[0]',
    'Laptop_1': 'diningTable/laptop[1]',
    'Fence_0': 'Floor/20cmHighWall100cm[0]',
    'Fence_1': 'Floor/20cmHighWall100cm[1]'
}

print("Certifique-se de que o CoppeliaSim está rodando com a cena T1.ttt aberta.")

In [ ]:
# Conectar ao CoppeliaSim
print("Conectando ao CoppeliaSim...")
connector = CoppeliaSimConnector()

if not connector.connect():
    raise RuntimeError("Falha na conexão com CoppeliaSim. Verifique se o simulador está rodando.")

# Descobrir objetos na cena
object_handles = connector.discover_objects(DEFAULT_OBJECT_MAPPING_EX5_6)

if not object_handles:
    raise RuntimeError("Nenhum objeto encontrado. Verifique se a cena T1.ttt está carregada.")

print(f"\nObjetos descobertos: {list(object_handles.keys())}")

In [ ]:
# Verificar pose inicial do robô
robot_pose = connector.get_object_pose('Robot')

if robot_pose:
    position, orientation = robot_pose
    print("Pose inicial do robô:")
    print(f"Posição (x, y, z): [{position[0]:.3f}, {position[1]:.3f}, {position[2]:.3f}] metros")
    print(f"Orientação (rx, ry, rz): [{orientation[0]:.3f}, {orientation[1]:.3f}, {orientation[2]:.3f}] radianos")

    # Converter para graus para melhor visualização
    orientation_deg = np.rad2deg(orientation)
    print(f"Orientação (rx, ry, rz): [{orientation_deg[0]:.1f}°, {orientation_deg[1]:.1f}°, {orientation_deg[2]:.1f}°]")
else:
    print("Erro: Não foi possível obter a pose do robô")

### Cenário Inicial: Posição Inicial

![Posição Init](Screenshot-inicial_EX5_6.png)

*Figura: mostrando a posição mencionada*

In [1]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

print('Program started')

try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    # Handle para o ROBÔ
    robotname = 'PioneerP3DX'
    # The new API uses sim.getObject to get handles. The path starts with '/'
    robotHandle = sim.getObject('/' + robotname)

    # Handle para as juntas das RODAS
    l_wheel = sim.getObject('/' + robotname + '/leftMotor')
    r_wheel = sim.getObject('/' + robotname + '/rightMotor')

    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)

    # Inicia a simulação
    sim.startSimulation()
    sim.step()

    # Fazendo uma leitura do laser
    hokuyo_sensor = HokuyoSensorSim(sim, "/"+ robotname +"/fastHokuyo")
    initial_laser_data = hokuyo_sensor.getSensorData()
    draw_laser_data(initial_laser_data)

    # Posição inicial do robô
    pos = sim.getObjectPosition(robotHandle, sim.handle_world)
    print(f'Initial Robot Position: {pos}')

    # Dados do Pioneer
    L = 0.381  # Metros
    r = 0.0975 # Metros

    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) < 30:
        print(f"Simulation time: {sim_time:.2f} [s]")

        # Fazendo leitura do laser
        laser_data = hokuyo_sensor.getSensorData()

        # Velocidade básica (linear, angular)
        v = 0
        w = np.deg2rad(0)

        frente = int(len(laser_data) / 2)
        lado_direito = int(len(laser_data) * 1 / 4)
        lado_esquerdo = int(len(laser_data) * 3 / 4)

        # Lógica de desvio de obstáculo
        if laser_data[frente, 1] > 2:
            v = .5
            w = 0
        elif laser_data[lado_direito, 1] > 2:
            v = 0
            w = np.deg2rad(-30)
        elif laser_data[lado_esquerdo, 1] > 2:
            v = 0
            w = np.deg2rad(30)

        # Modelo cinemático
        wl = v / r - (w * L) / (2 * r)
        wr = v / r + (w * L) / (2 * r)

        # Enviando velocidades (não precisa mais de opmode)
        sim.setJointTargetVelocity(l_wheel, wl)
        sim.setJointTargetVelocity(r_wheel, wr)

        sim.step()

    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(r_wheel, 0)
    sim.setJointTargetVelocity(l_wheel, 0)

except Exception as e:
    print(f"An error occurred: {e}")

# Parando a simulação
sim.stopSimulation()

print('Program ended')

Program started


KeyboardInterrupt: 